In [1]:
#!/usr/bin/env python3
"""
Read SO-CI output from Molpro.
Assign omegas, including parity. 
Also show composition of a level, or distribution of a term.
C2v symmetry (4 irreps) is assumed. 
KKI March 2023: include dipole moment
"""
import re, sys, copy, glob, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '../atomic_SOC')
import molpro_subs as mpr
import chem_subs as chem

pd.set_option('display.width', 1000)

In [2]:
fsoci = 'ac5z_hybB_r2p2444_lz.pro'
print('Will read SO-CI info from file: {:s}'.format(fsoci))

Will read SO-CI info from file: ac5z_hybB_r2p2444_lz.pro


In [3]:
# Read CASSCF
PG = mpr.read_point_group(fsoci)
print('Point group is', PG)
crd, lineno_crd = mpr.read_coordinates(fsoci, linenum=True)
if isinstance(lineno_crd, list):
    # take last geometry
    crd = crd[-1]
    lineno_crd = lineno_crd[-1]
# get diatomic bond length
G = chem.Geometry(crd, intype='DataFrame', units='bohr')
G.toAngstrom()
R = np.round(G.distance(0, 1), 6)  # round the bond length to 6 digits
print('Bond length = {:.4f}'.format(R))
caslist, lineno_cas = mpr.readMULTI(fsoci, PG=PG, linenum=True)
CAS = caslist[-1]   # assume the last CASSCF to be the relevant one

oldcas = CAS.results.copy()
for Spin in sorted(set(oldcas.Spin)):
    nspin = len(oldcas[oldcas.Spin == Spin])
    print('{:d} {:s}s'.format(nspin, Spin))
print('Active space = {:d}/{:d}'.format(CAS.nactel(), CAS.nactorb()))
CAS.results = mpr.relabel_CAS_by_energy(CAS.results)
# Note any changes in CAS labeling
diflbl = (oldcas.Label != CAS.results.Label).values
if diflbl.any():
    dflbl = oldcas[['Label', 'Term']].copy()
    dflbl['NewLabel'] = CAS.results.Label
    print('Some CAS labels changed:')
    display(dflbl)
# check for dynamical weights
rx_dynw = re.compile('[~!]*dynw,(\d+)')
dynw = 0
with open(fsoci, 'r') as F:
    for line in F:
        m = rx_dynw.search(line)
        if m:
            dynw = int(m.group(1))
            print('Dynamical weighting with dynw = {:d}'.format(dynw))
if not dynw:
    print('Uniform weighting')

Point group is C2v
Bond length = 2.2444
35 Doublets
15 Quartets
Active space = 19/11
Dynamical weighting with dynw = 8
Dynamical weighting with dynw = 8


In [4]:
# count the terms that are included in the calculation
from collections import Counter
for spin, grp in CAS.results.groupby('Spin'):
    print(spin)
    print(Counter(grp.Term.tolist()))
#CAS.results

Doublet
Counter({'2Π': 12, '2Δ': 10, '2Σ+': 5, '2Φ': 4, '2Γ': 2, '2Σ-': 2})
Quartet
Counter({'4Π': 6, '4Δ': 4, '4Φ': 2, '4Σ-': 2, '4Σ+': 1})


In [5]:
casdf = CAS.results[['Irrep', 'Label', 'Energy', 'Term']].copy()
casdf['S'] = [chem.spinname(x)-1 for x in CAS.results.Spin]
casdf['Lz'] = np.round(np.sqrt(np.abs(CAS.results.LzLz)), 0).astype(int)
spin = sorted(set(casdf.S))
irreps = sorted(set(casdf.Irrep))
lzvals = sorted(set(casdf.Lz))

In [6]:
# check for proper pairing of degenerate states
broken = False
for grp, dfg in casdf.groupby(['S', 'Lz']):
    if grp[1] == 0:
        # ignore Sigma states
        continue
    # number in each irrep should be equal
    grpi = dfg.groupby('Irrep')
    lens = grpi.size().values
    if lens[0] != lens[1]:
        print('Broken pair somewhere')
        display(dfg.sort_values('Energy'))
        broken = True
if not broken:
    print('All CASSCF state pairs are closed')

All CASSCF state pairs are closed


In [7]:
# check for non-integer Lz values
cruft = casdf.Lz - np.round(casdf.Lz, 0)
if cruft.any():
    print('*** There are non-integer values of Lz')
else:
    print('Lz values look OK')

Lz values look OK


In [8]:
# order the CASSCF states by energy within each (S, irrep) group
newdf = pd.DataFrame(columns=casdf.columns)
for lbl, dg in casdf.groupby(['S', 'Irrep']):
    dt = dg.copy().sort_values('Energy').reset_index(drop=True)
    #display(dt)
    newdf = newdf.append(dt)

In [9]:
# read MRCI
cilist, lineno_ci = mpr.readMRCI(fsoci, linenum=True)   # probably many
for m in cilist:
    m.transfer_lz(CAS.results)
mrci = [mpr.MRCIstate(row) for m in cilist for (irow, row) in m.results.iterrows()]
dfci = mpr.combineMRCI(cilist)

In [10]:
# check for erroneously repeated CI roots
tol = 1.e-6
for irrep, grp in dfci.sort_values('Energy').groupby(['Spin', 'Irrep']):
    e = grp.Energy.values
    de = e[1:] - e[:-1]
    smal = np.abs(de) < tol
    for i, s in enumerate(smal):
        if s:
            print('Warning: closely repeated root in MRCI')
            display(grp.iloc[[i,i+1]])
print('Checking for discrepancies between reference energy and CASSCF energy')
dfcheck = dfci[['Spin', 'Irrep', 'Label', 'Edav', 'Eref']].copy()
ecas = []
for i, row in dfcheck.iterrows():
    ecas.append(CAS.results[(CAS.results.Label == row.Label) & (CAS.results.Spin == row.Spin)].Energy.values[0])
dfcheck['CAS'] = ecas
dfcheck['diff'] = np.round(dfcheck.CAS - dfcheck.Eref, 6)
dfbad = dfcheck[np.abs(dfcheck['diff']) > 0.1].sort_values(['Spin', 'Irrep', 'Label'])
if len(dfbad):
    display(dfbad)
else:
    print('\t--looks good')

Checking for discrepancies between reference energy and CASSCF energy
	--looks good


In [11]:
# read SO-CI
SOCI = mpr.fullmatSOCI(fsoci, hybrid=True)
dfterms = SOCI.average_terms()
ok = SOCI.assign_omega(ordering='up', failure='OK')
if not ok:
    print('\n*** OMEGA ASSIGNMENTS FAILED ***')

Computational group = C2v
CASSCF states:
    35 Doublet
    15 Quartet
Replacing MRCI+Q energies by HLSDIAG values
*** Warning: "130 SO states but only 84 unique basis states" in parse_SOmatrix()
Target omega counts: {0.5: 48, 1.5: 40, 2.5: 26, 3.5: 12, 4.5: 4}
SO level (state) counts:
	1/2	24	(48)
	3/2	20	(40)
	5/2	13	(26)
	7/2	6	(12)
	9/2	2	(4)
130 states
65 levels


In [12]:
if not ok:
    thresh = 2.e-5 # for energies
    wthr = 1.e-3   # for leading-term weights
    pd.set_option('display.max_rows', 500)
    print('*** Attempt to assign Omegas by interpolation!  ***')
    print('--- Using energy only ---')
    datfile = r'C:\Users\irikura\OneDrive - NIST\Karl\PtH_anion\guiding_scan\guiding_pots.tsv'
    print(f'Using PEC data file {datfile}')
    dfPEC = pd.read_csv(datfile, sep='\t')
    # expect a column 'R' followed by columns with state labels
    # generate interpolations
    from collections import Counter
    Einterp = []
    Ominterp = []
    re_om = re.compile('(\d)[-+]?$')
    x = dfPEC.R.values
    states = []
    for st in dfPEC.columns[1:]:
        y = dfPEC[st].values
        fPEC = chem.fit_diatomic_potential(x, y)
        E = float(fPEC(R))
        m = re_om.search(st)
        Om = int(m.group(1))
        states.append(st)
        Einterp.append(E)
        Ominterp.append(Om)
    dfinterp = pd.DataFrame({'Label': states, 'Om': Ominterp, 'E': Einterp}).sort_values('E').reset_index(drop=True)
    # add column for energy increments
    evals = dfinterp.E.values
    incr = [np.nan] + list(evals[1:] - evals[:-1])
    dfinterp['incrE'] = np.round(incr, 6)
    dfinterp['used'] = False
    print('\nInterpolated states (expectation):')
    display(dfinterp)
    icount = {}
    for om, v in Counter(dfinterp.Om).items():
        if om == 0:
            icount[om] = v
        else:
            icount[om] = v * 2
    icount = Counter(icount)
    print('Target (correct) counts:', icount)
    
    # Do not re-sort dfso by energy because ordering of degen levels can get scrambled
    dfso = SOCI.dfso.sort_values('E').reset_index(drop=True)  # to be assigned
    dfso['Ω'] = None
    dfso['label'] = None
    dfso['Olbl'] = None
    evals = dfso.E.values
    incr = [np.nan] + list(evals[1:] - evals[:-1])
    dfso['incrE'] = np.round(incr, 6)
    wts = dfso.wt.values
    dwt = [np.nan] + list(wts[1:] - wts[:-1])
    dfso['wtdiff'] = np.round(np.abs(dwt), 5)
    nstates = len(dfso)
    print('\nActual states to be assigned:')
    display(dfso)
   
    # Since usual Omega assignment failed, do not assume that the term symbols are reliable
    def assignstate():
        # use globals
        dfso.loc[i, 'Olbl'] = jow.Label
        dfso.loc[i, 'Ω'] = jow.Om
        dfinterp.loc[j, 'used'] = True
        print(f'assign state {i} with {jow.Label}')
        if jow.Om > 0:
            # also assign its twin (should be the next state by energy)
            dfso.loc[i+1, 'Olbl'] = jow.Label
            dfso.loc[i+1, 'Ω'] = jow.Om                    
            print(f'\tpair {i+1} with {jow.Label}')
        return

    print('Start with biggest gaps:')
    # look for big gaps, as more reliable than small gaps
    dfinterp = dfinterp.sort_values('incrE', ascending=False)
    for i, row in dfso.sort_values('incrE', ascending=False).iterrows():
        if row.incrE < 2 * thresh:
            # too small to trust
            continue
        #display(row.to_frame().T)
        for j, jow in dfinterp.iterrows():
            if jow.incrE < 2 * thresh:
                # gap too small to trust
                continue
            if jow.used:
                # already matched to an actual state
                continue
            if abs(row.incrE - jow.incrE) < thresh:
                de = abs(row.E - jow.E)
                if de < thresh:
                    # this looks like a match; check for accidental degeneracy
                    #if (i+1 < nstates) and (dfso.loc[i+1, 'incrE'] < thresh):
                    #    # next level is close; it might be the twin
                    #    if (i+2 < nstates) and (dfso.loc[i+2, 'incrE'] > thresh):
                    #        # looks like an accidental degeneracy (3 levels)
                    #        break
                    # this is a match; assign it
                    assignstate()
    dfinterp = dfinterp.sort_values('E')
    display(dfso[dfso.Ω.isnull()])
    counts = Counter(dfso.Ω)
    print('current counts:', counts)
    print('missing counts:', icount - counts)
    
    # drop used levels from dfinterp
    dfinterp = dfinterp[dfinterp.used == False]

    print('\nLook for close energy matches:')
    for i, row in dfso.iterrows():
        if row.Ω is not None:
            # already assigned
            continue
        for j, jow in dfinterp.iterrows():
            if jow.used:
                # row already matched
                continue
            dE = abs(row.E - jow.E)
            if dE < thresh:
                # a match
                assignstate()
    display(dfso[dfso.Ω.isnull()])
    counts = Counter(dfso.Ω)
    print('current counts:', counts)
    print('missing counts:', icount - counts)
    dfinterp = dfinterp[dfinterp.used == False]

    if (len(dfinterp) == 0) and (sum((icount - counts).values()) == 0):
        print('\nAll states assigned!')
        # drop unneeded columns
        dfdups = dfso[['E', 'cm-1', 'Ω', 'exc', 'Olbl']]
        df = mpr.average_SO_levels(dfdups, be_same=['Ω'])
        #df['Olbl'] = chem.enumerative_prefix([s.split('_')[1] for s in df.label])
        display(df)
        # copy into the SOCI object
        SOCI.dfso = df

In [13]:
dfterms['Ecm'] = np.round((dfterms.Edav - dfterms.Edav.min()) * chem.AU2CM, 1)
dfterms

,Term,dipZ,Edav,idx,ecm,Ecm
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.0
1,(1)2Σ+,-0.686412,-119.783698,[0],80.5,80.5
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,1480.7
3,(1)4Δ,-0.161305,-119.730043,"[48, 36]",11856.4,11856.4
4,(2)2Σ+,-0.014132,-119.727746,[1],12360.4,12360.4
5,(1)4Π,-0.096289,-119.727039,"[42, 38]",12515.7,12515.7
6,(2)2Δ,0.101961,-119.718361,"[30, 6]",14420.2,14420.2
7,(2)2Π,0.095556,-119.716350,"[12, 20]",14861.5,14861.5
8,(1)4Σ-,-0.142579,-119.715854,[46],14970.4,14970.4
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,15301.1


In [14]:
# Is this a hybrid calculation (prepared by build_hybrid_soci_input.ipynb)?
ccterms = []  # list of input CCSD(T) terms
rx_hyb = re.compile('HLSDIAG\(.+\s+!\s*.*(ccsd|anchored|shifted|input)')
is_hybrid = False
with open(fsoci, 'r', encoding='utf8') as F:
    for line in F:
        if rx_hyb.search(line):
            is_hybrid = True
            if 'input' in line:
                # an anchor term; extract its label
                words = line.split()
                if words[2] not in ccterms:
                    ccterms.append(words[2])

In [15]:
# transpose absolute energies and copy to clipboard
dfcp = SOCI.dfso[['Olbl', 'E', '<i|z|i>']].copy().sort_values('Olbl').set_index('Olbl')
dfcp.rename(columns={'E': f'{R}'}, inplace=True)   # put bond length in that position for pasting to Excel
dfcp.T.to_clipboard()
if is_hybrid:
    print(f'Hybrid SO-CI energies for R={R} copied to clipboard, for pasting into Excel')
else:
    print(f'Standard SO-CI energies for R={R} copied to clipboard, for pasting into Excel')

Hybrid SO-CI energies for R=2.2444 copied to clipboard, for pasting into Excel


In [16]:
dfcp

,2.2444,<i|z|i>
Olbl,,
(1)1/2,-119.854930,-0.669626
(1)3/2,-119.854576,-0.604866
(1)5/2,-119.860885,-0.557529
(1)7/2,-119.813865,-0.211859
(1)9/2,-119.807160,-0.108048
...,...,...
(8)3/2,-119.773749,-0.051672
(8)5/2,-119.759103,0.061946
(9)1/2,-119.771077,0.074910


### Term composition of some levels

In [17]:
olabels = ['(1)1/2', '(1)3/2', '(1)5/2']
for olabel in olabels:
    ilev = SOCI.dfso[SOCI.dfso.Olbl == olabel].index[0]
    #print('ilvel =', ilev)
    print('Composition of level #{:d}:  "{:s}" or "{:s}"'.format(ilev, SOCI.dfso.loc[ilev].Olbl,
                                                             SOCI.dfso.loc[ilev].label))
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6, normalize=True)
    wtcol = dfterm.columns[-1]
    display(dfterm.sort_values(wtcol, ascending=False))
    print('Sum of weights = {:.3f}'.format(dfterm[wtcol].sum()))
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    print('Sum of weights from CC terms = {:.3f}\n'.format(ccsum))

Composition of level #2:  "(1)1/2" or "(1)2Σ+_1/2"
Eso[2] = -2863.8 cm-1


,Term,dipZ,Edav,idx,ecm,Ecm,wt_02
1,(1)2Σ+,-0.686412,-119.783698,[0],80.5,80.5,0.686890
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,1480.7,0.297112
5,(1)4Π,-0.096289,-119.727039,"[42, 38]",12515.7,12515.7,0.009636
8,(1)4Σ-,-0.142579,-119.715854,[46],14970.4,14970.4,0.001485
15,(1)2Σ-,0.067737,-119.705673,[27],17204.9,17204.9,0.000971
10,(3)2Σ+,-0.079039,-119.713049,[2],15586.1,15586.1,0.000963
12,(3)2Π,0.097561,-119.709794,"[13, 21]",16300.4,16300.4,0.000769
7,(2)2Π,0.095556,-119.716350,"[12, 20]",14861.5,14861.5,0.000600
23,(4)2Σ+,0.120868,-119.656262,[3],28049.4,28049.4,0.000351
20,(3)4Π,0.003020,-119.670024,"[40, 44]",25028.9,25028.9,0.000313


Sum of weights = 1.000
Sum of weights from CC terms = 0.994

Composition of level #4:  "(1)3/2" or "(1)2Π_3/2"
Eso[4] = -2785.9 cm-1


,Term,dipZ,Edav,idx,ecm,Ecm,wt_04
2,(1)2Π,-0.652485,-119.777318,"[11, 19]",1480.7,1480.7,0.706705
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.0,0.275432
8,(1)4Σ-,-0.142579,-119.715854,[46],14970.4,14970.4,0.005436
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,15301.1,0.003421
12,(3)2Π,0.097561,-119.709794,"[13, 21]",16300.4,16300.4,0.002956
26,(5)2Δ,-0.010376,-119.649384,"[9, 33]",29559.0,29559.0,0.001437
13,(2)4Π,0.147435,-119.708154,"[39, 43]",16660.4,16660.4,0.001115
20,(3)4Π,0.003020,-119.670024,"[40, 44]",25028.9,25028.9,0.000850
16,(3)2Δ,0.332825,-119.703829,"[31, 7]",17609.7,17609.7,0.000631
22,(4)2Δ,-0.160284,-119.657767,"[8, 32]",27719.0,27719.0,0.000373


Sum of weights = 1.000
Sum of weights from CC terms = 0.982

Composition of level #0:  "(1)5/2" or "(1)2Δ_5/2"
Eso[0] = -4170.8 cm-1


,Term,dipZ,Edav,idx,ecm,Ecm,wt_00
0,(1)2Δ,-0.588172,-119.784064,"[29, 5]",0.0,0.0,0.982950
13,(2)4Π,0.147435,-119.708154,"[39, 43]",16660.4,16660.4,0.005210
9,4Φ,-0.110737,-119.714348,"[41, 45]",15301.1,15301.1,0.004648
20,(3)4Π,0.003020,-119.670024,"[40, 44]",25028.9,25028.9,0.002686
28,(2)2Φ,-0.046586,-119.648791,"[18, 26]",29689.1,29689.1,0.001745
6,(2)2Δ,0.101961,-119.718361,"[30, 6]",14420.2,14420.2,0.000975
14,(1)2Φ,0.069270,-119.706699,"[17, 25]",16979.7,16979.7,0.000948
5,(1)4Π,-0.096289,-119.727039,"[42, 38]",12515.7,12515.7,0.000426
26,(5)2Δ,-0.010376,-119.649384,"[9, 33]",29559.0,29559.0,0.000187
16,(3)2Δ,0.332825,-119.703829,"[31, 7]",17609.7,17609.7,0.000096


Sum of weights = 1.000
Sum of weights from CC terms = 0.983



### Distribution of a term among levels

In [18]:
#term = '(1)1Σ+'
term = '(1)2Δ'
if is_hybrid:
    typ = 'hybrid'
else:
    typ = 'standard'
print('Distribution of term "{:s}" among {:s} levels'.format(term, typ))
print(f'R = {R}')
df = SOCI.level_contributions_from_term(term, thr=1.e-6, normalize=True)
df['prod'] = np.round(df.exc * df[term], 1)
ebar = df['prod'].sum() / df[term].sum()
print('Weighted mean energy of {:s} = {:.1f} cm-1'.format(term, ebar))
#display(df.sort_values(term, ascending=False))
display(df[df[term] > 0.0005].sort_values('E').style.format({term: '{:.4f}'}))
print('Total weight = {:.4f}'.format(df[term].sum()))

Distribution of term "(1)2Δ" among hybrid levels
R = 2.2444
Weighted mean energy of (1)2Δ = 4170.9 cm-1


,E,cm-1,Ω,label,exc,g,Olbl,(1)2Δ,prod
0,-119.860885,-4170.780000,2.500000,(1)2Δ_5/2,0.000000,2,(1)5/2,0.4915,0.000000
4,-119.854576,-2785.940000,1.500000,(1)2Π_3/2,1384.840000,2,(1)3/2,0.1377,190.700000
9,-119.816169,5643.380000,1.500000,(1)2Δ_3/2,9814.160000,2,(2)3/2,0.3348,3286.300000
20,-119.804187,8273.070000,2.500000,(2)2Δ_5/2,12443.850000,2,(3)5/2,0.0024,29.400000
24,-119.802482,8647.240000,1.500000,(1)4Σ-_3/2,12818.020000,2,(4)3/2,0.0086,110.300000
33,-119.794782,10337.330000,1.500000,(2)4Π_3/2,14508.110000,2,(5)3/2,0.0015,21.400000
36,-119.792809,10770.260000,1.500000,4Σ+_3/2,14941.040000,2,(6)3/2,0.0011,16.000000
38,-119.792630,10809.560000,2.500000,(2)4Π_5/2,14980.340000,2,(4)5/2,0.0008,11.400000
42,-119.789376,11523.830000,2.500000,(3)2Δ_5/2,15694.610000,2,(5)5/2,0.0008,12.200000
46,-119.775777,14508.390000,2.500000,(2)2Δ_5/2,18679.170000,2,(6)5/2,0.0006,12.000000


Total weight = 1.0000


### Contribution of CCSD(T) terms to all levels

In [19]:
# Get contributions of CC terms to all levels
# Loop over levels to get the normalization right
dfcc = SOCI.dfso.copy()
ccsums = []
for ilev in dfcc.index:
    dfci, dfterm = SOCI.composition_of_level(ilev, thr=1.e-6,
                                normalize=True, silent=True)
    wtcol = dfterm.columns[-1]
    ccsum = dfterm[dfterm.Term.isin(ccterms)][wtcol].sum()
    ccsums.append(ccsum)
dfcc['CCwt'] = ccsums
fmt = {chem.OMEGA: '{:.1f}', 'exc': '{:.2f}', 'CCwt': '{:.3f}'}
display(dfcc[dfcc.columns[2:]].style.format(fmt))
print('Total of CCwt column = {:.3f}'.format(dfcc.CCwt.sum()))
print('The CC terms are', ccterms)

,Ω,term,wt,label,Nr,exc,,g,Olbl,CCwt
0,2.5,(1)2Δ,0.491530,(1)2Δ_5/2,"[1, 2]",0.00,-0.557529,2,(1)5/2,0.983
2,0.5,(1)2Σ+,0.686890,(1)2Σ+_1/2,"[3, 4]",1307.02,-0.669626,2,(1)1/2,0.994
4,1.5,(1)2Π,0.353423,(1)2Π_3/2,"[5, 6]",1384.84,-0.604866,2,(1)3/2,0.982
6,0.5,(1)2Π,0.297393,(1)2Π_1/2,"[7, 8]",9730.07,-0.474625,2,(2)1/2,0.893
9,1.5,(1)2Δ,0.334880,(1)2Δ_3/2,"[9, 10]",9814.16,-0.562612,2,(2)3/2,0.952
10,3.5,(1)4Δ,0.431232,(1)4Δ_7/2,"[11, 12]",10319.77,-0.211859,2,(1)7/2,0.862
12,2.5,(1)4Δ,0.283580,(1)4Δ_5/2,"[13, 14]",10523.49,-0.214297,2,(2)5/2,0.858
14,1.5,(1)4Π,0.199769,(1)4Π_3/2,"[15, 16]",10897.94,-0.204681,2,(3)3/2,0.717
17,0.5,(1)4Σ-,0.318519,(1)4Σ-_1/2,"[17, 18]",11298.31,-0.218353,2,(3)1/2,0.451
18,4.5,4Φ,0.483418,4Φ_9/2,"[19, 20]",11791.47,-0.108048,2,(1)9/2,0.000


Total of CCwt column = 13.000
The CC terms are ['(1)2Σ+', '(1)2Δ', '(1)2Π', '(1)4Δ', '(1)4Π']


In [20]:
df = SOCI.dfso[SOCI.dfso['term'] == '4Φ']
display(df)
print('wt sum = {:.3f}'.format(df.wt.sum()))

,E,cm-1,Ω,term,wt,label,Nr,exc,<i|z|i>,g,Olbl
18,-119.807160,7620.69,4.5,4Φ,0.483418,4Φ_9/2,"[19, 20]",11791.47,-0.108048,2,(1)9/2
26,-119.800519,9078.20,3.5,4Φ,0.241250,4Φ_7/2,"[27, 28]",13248.98,-0.012597,2,(2)7/2
56,-119.766861,16465.13,2.5,4Φ,0.265470,4Φ_5/2,"[57, 58]",20635.91,-0.052543,2,(7)5/2
65,-119.761525,17636.39,3.5,4Φ,0.183775,4Φ_7/2,"[65, 66]",21807.17,0.039794,2,(4)7/2
68,-119.758537,18292.11,1.5,4Φ,0.145333,4Φ_3/2,"[69, 70]",22462.89,0.050009,2,(11)3/2


wt sum = 1.319


In [21]:
SOCI.dfso[(SOCI.dfso.exc > 21000) & (SOCI.dfso.exc < 30000)]

,E,cm-1,Ω,term,wt,label,Nr,exc,<i|z|i>,g,Olbl
60,-119.765074,16857.30,1.5,(1)4Σ-,0.288440,(1)4Σ-_3/2,"[61, 62]",21028.08,0.000860,2,(10)3/2
63,-119.763778,17141.79,0.5,4Σ+,0.150226,4Σ+_1/2,"[63, 64]",21312.57,-0.015902,2,(11)1/2
65,-119.761525,17636.39,3.5,4Φ,0.183775,4Φ_7/2,"[65, 66]",21807.17,0.039794,2,(4)7/2
66,-119.759103,18167.97,2.5,(1)2Φ,0.178656,(1)2Φ_5/2,"[67, 68]",22338.75,0.061946,2,(8)5/2
68,-119.758537,18292.11,1.5,4Φ,0.145333,4Φ_3/2,"[69, 70]",22462.89,0.050009,2,(11)3/2
70,-119.758012,18407.30,0.5,4Σ+,0.180335,4Σ+_1/2,"[71, 72]",22578.08,0.213261,2,(12)1/2
72,-119.754019,19283.60,0.5,(2)4Δ,0.151676,(2)4Δ_1/2,"[73, 74]",23454.38,0.118742,2,(13)1/2
74,-119.749154,20351.34,1.5,(2)4Σ-,0.113665,(2)4Σ-_3/2,"[75, 76]",24522.12,0.058934,2,(12)3/2
76,-119.748376,20522.23,0.5,(2)2Σ+,0.195575,(2)2Σ+_1/2,"[77, 78]",24693.01,0.012341,2,(14)1/2
78,-119.743510,21590.27,0.5,(2)2Π,0.121155,(2)2Π_1/2,"[79, 80]",25761.05,0.062539,2,(15)1/2
